# Google Finance Tool

This notebook provides a quick overview for getting started with [`Google Finance Tool`](/docs/integrations/tools/). For detailed documentation of all `Google Finance Tool` features and configurations head to the [API reference](https://v03.api.js.langchain.com/modules/_langchain_community.tools_google_finance.html).

This tool uses the [SerpApi Google Finance API](https://serpapi.com/google-finance-api), so you will need a [SerpApi API key](https://serpapi.com/dashboard).

## Overview

### Integration details

| Class | Package | [PY support](https://python.langchain.com/docs/integrations/tools/google_finance/) |  Package latest |
| :--- | :--- | :---: | :---: |
| [`SERPGoogleFinanceAPITool`](https://v03.api.js.langchain.com/modules/_langchain_community.tools_google_finance.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ | ![NPM - Version](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## Setup

The integration lives in the `@langchain/community` package.

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core
</Npm2Yarn>
```

### Credentials

```typescript
process.env.SERPAPI_API_KEY="your-serpapi-api-key"
```

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability:

```typescript
process.env.LANGCHAIN_TRACING_V2="true"
process.env.LANGCHAIN_API_KEY="your-api-key"
```

## Instantiation

You can import and instantiate an instance of the `SERPGoogleFinanceAPITool` tool like this:

In [ ]:
import { SERPGoogleFinanceAPITool } from "@langchain/community/tools/google_finance";

const tool = new SERPGoogleFinanceAPITool({
  // optional, either pass in your api key here or set it as an environment variable
  apiKey: "your-serpapi-api-key",
});

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

The input is a query string for anything you might search for on Google Finance (e.g. GOOG:NASDAQ):

In [ ]:
await tool.invoke("GOOG:NASDAQ")

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

We can also invoke the tool with a model-generated `ToolCall`, in which case a `ToolMessage` will be returned:

In [ ]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "What is the price of GOOG:NASDAQ?"
  },
  id: "1",
  name: tool.name,
  type: "tool_call",
}
await tool.invoke(modelGeneratedToolCall)

## Chaining

We can use our tool in a chain by first binding it to a [tool-calling model](/docs/how_to/tool_calling/) and then calling it:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [ ]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
})

In [ ]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("What is the price of GOOG:NASDAQ?");

In [ ]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

## Agents

For guides on how to use LangChain tools in agents, see the [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) docs.

## API reference

For detailed documentation of all `Google Finance Tool` features and configurations, head to the [API reference](https://v03.api.js.langchain.com/modules/_langchain_community.tools_google_finance.html).